In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data

In [2]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

#data_path = "../dataset/TSmall"
#postfix = "TSmall"
#dim = 70
data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))



ind = int(0.8 * len(index))

s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_triu_norm, mu, std = AbstractDataset.normalize(s_triu)


s_train, p_train, s_test, p_test = split(s_triu_norm, p_triu, ind)

# Model

In [3]:
filepath = "../../models/" + "TSmall_sto3gmodelTSmall_sto3g_250-150-50+triu.h5"
model = keras.models.load_model(filepath)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 351)               123552    
_________________________________________________________________
dense_1 (Dense)              (None, 601)               211552    
_________________________________________________________________
dense_2 (Dense)              (None, 501)               301602    
_________________________________________________________________
dense_3 (Dense)              (None, 401)               201302    
_________________________________________________________________
dense_4 (Dense)              (None, 351)               141102    
Total params: 979,110
Trainable params: 979,110
Non-trainable params: 0
_________________________________________________________________


# Analysis Prerequisits

In [4]:
from SCFInitialGuess.utilities.analysis import make_results_str, measure_all_quantities

N_ELECTRONS = 30


In [5]:
from SCFInitialGuess.utilities.dataset import StaticDataset

dataset = StaticDataset(
    train=(s_train, p_train),
    validation=(None, None),
    test=(s_test, p_test),
    mu=mu,
    std=std
)

In [6]:
def format_results(result):
    if isinstance(result, list):
        out = list(map(
            lambda x: "{:0.5E} +- {:0.5E}".format(*x),
            result
        ))
        out = "\n".join(out)
    else:
        out =  "{:0.5E} +- {:0.5E}".format(*result)
    return out

# Calculate Guesses 

In [7]:
from SCFInitialGuess.utilities.dataset import make_matrix_batch
from SCFInitialGuess.nn.post_processing import multi_mc_wheeny
p_nn = model.predict(s_test)

s_raw = make_matrix_batch(s_triu[ind:], dim, is_triu=True)

p_batch = make_matrix_batch(p_nn, dim, True).astype("float64")
p_mcw1 = np.array(list(map(lambda x: multi_mc_wheeny(x[0], x[1], n_max=1), zip(p_batch, s_raw)))).astype("float64")
p_mcw5 = np.array(list(map(lambda x: multi_mc_wheeny(x[0], x[1], n_max=5), zip(p_batch, s_raw)))).astype("float64")

# Analysis McW1

In [8]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer

print(make_results_str(measure_all_quantities(
    p_mcw1,
    dataset,
    molecules[ind:],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-11-05 23:40:31: Iteration calculation: 0
[ ] 2018-11-05 23:40:32: Iteration calculation: 1
[ ] 2018-11-05 23:40:32: Iteration calculation: 2
[ ] 2018-11-05 23:40:32: Iteration calculation: 3
[ ] 2018-11-05 23:40:33: Iteration calculation: 4
[ ] 2018-11-05 23:40:33: Iteration calculation: 5
[ ] 2018-11-05 23:40:33: Iteration calculation: 6
[ ] 2018-11-05 23:40:33: Iteration calculation: 7
[ ] 2018-11-05 23:40:33: Iteration calculation: 8
[ ] 2018-11-05 23:40:33: Iteration calculation: 9
[ ] 2018-11-05 23:40:34: Iteration calculation: 10
[ ] 2018-11-05 23:40:34: Iteration calculation: 11
[ ] 2018-11-05 23:40:34: Iteration calculation: 12
[ ] 2018-11-05 23:40:34: Iteration calculation: 13
[ ] 2018-11-05 23:40:35: Iteration calculation: 14
[ ] 2018-11-05 23:40:35: Iteration calculation: 15
[ ] 2018-11-05 23:40:35: Iteration calculation: 16
[ ] 2018-11-05 23:40:36: Iteration calculation: 17
[ ] 2018-11-05 23:40:36: Iteration calculation: 18
[ ] 2018-11-05 23:40:36: Iteration calcul

[ ] 2018-11-05 23:41:03: Iteration calculation: 160
[ ] 2018-11-05 23:41:03: Iteration calculation: 161
[ ] 2018-11-05 23:41:04: Iteration calculation: 162
[ ] 2018-11-05 23:41:04: Iteration calculation: 163
[ ] 2018-11-05 23:41:04: Iteration calculation: 164
[ ] 2018-11-05 23:41:04: Iteration calculation: 165
[ ] 2018-11-05 23:41:04: Iteration calculation: 166
[ ] 2018-11-05 23:41:04: Iteration calculation: 167
[ ] 2018-11-05 23:41:04: Iteration calculation: 168
[ ] 2018-11-05 23:41:05: Iteration calculation: 169
[ ] 2018-11-05 23:41:05: Iteration calculation: 170
[ ] 2018-11-05 23:41:05: Iteration calculation: 171
[ ] 2018-11-05 23:41:05: Iteration calculation: 172
[ ] 2018-11-05 23:41:05: Iteration calculation: 173
[ ] 2018-11-05 23:41:05: Iteration calculation: 174
[ ] 2018-11-05 23:41:06: Iteration calculation: 175
[ ] 2018-11-05 23:41:06: Iteration calculation: 176
[ ] 2018-11-05 23:41:06: Iteration calculation: 177
[ ] 2018-11-05 23:41:06: Iteration calculation: 178
[ ] 2018-11-

In [21]:
7 / 201

0.03482587064676617

In [9]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics

print("--- Iterations Damped ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_damping,
   p_mcw1.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-05 23:41:09: Iteration calculation: 0
[ ] 2018-11-05 23:41:10: Iteration calculation: 1
[ ] 2018-11-05 23:41:10: Iteration calculation: 2
[ ] 2018-11-05 23:41:10: Iteration calculation: 3
[ ] 2018-11-05 23:41:10: Iteration calculation: 4
[ ] 2018-11-05 23:41:10: Iteration calculation: 5
[ ] 2018-11-05 23:41:10: Iteration calculation: 6
[ ] 2018-11-05 23:41:10: Iteration calculation: 7
[ ] 2018-11-05 23:41:11: Iteration calculation: 8
[ ] 2018-11-05 23:41:11: Iteration calculation: 9
[ ] 2018-11-05 23:41:11: Iteration calculation: 10
[ ] 2018-11-05 23:41:11: Iteration calculation: 11
[ ] 2018-11-05 23:41:11: Iteration calculation: 12
[ ] 2018-11-05 23:41:11: Iteration calculation: 13
[ ] 2018-11-05 23:41:12: Iteration calculation: 14
[ ] 2018-11-05 23:41:12: Iteration calculation: 15
[ ] 2018-11-05 23:41:12: Iteration calculation: 16
[ ] 2018-11-05 23:41:12: Iteration calculation: 17
[ ] 2018-11-05 23:41:12: Iteration calculation: 18
[ ] 2018-11-05 23:41:13: Iteration calcul

[ ] 2018-11-05 23:41:34: Iteration calculation: 161
[ ] 2018-11-05 23:41:34: Iteration calculation: 162
[ ] 2018-11-05 23:41:34: Iteration calculation: 163
[ ] 2018-11-05 23:41:34: Iteration calculation: 164
[ ] 2018-11-05 23:41:34: Iteration calculation: 165
[ ] 2018-11-05 23:41:34: Iteration calculation: 166
[ ] 2018-11-05 23:41:35: Iteration calculation: 167
[ ] 2018-11-05 23:41:35: Iteration calculation: 168
[ ] 2018-11-05 23:41:35: Iteration calculation: 169
[ ] 2018-11-05 23:41:35: Iteration calculation: 170
[ ] 2018-11-05 23:41:35: Iteration calculation: 171
[ ] 2018-11-05 23:41:35: Iteration calculation: 172
[ ] 2018-11-05 23:41:35: Iteration calculation: 173
[ ] 2018-11-05 23:41:36: Iteration calculation: 174
[ ] 2018-11-05 23:41:36: Iteration calculation: 175
[ ] 2018-11-05 23:41:36: Iteration calculation: 176
[ ] 2018-11-05 23:41:36: Iteration calculation: 177
[ ] 2018-11-05 23:41:36: Iteration calculation: 178
[ ] 2018-11-05 23:41:36: Iteration calculation: 179
[ ] 2018-11-

In [10]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics

print("--- Iterations DIIS ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_diis,
    p_mcw1.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-05 23:41:39: Iteration calculation: 0
[ ] 2018-11-05 23:41:39: Iteration calculation: 1
[ ] 2018-11-05 23:41:40: Iteration calculation: 2
[ ] 2018-11-05 23:41:40: Iteration calculation: 3
[ ] 2018-11-05 23:41:40: Iteration calculation: 4
[ ] 2018-11-05 23:41:40: Iteration calculation: 5
[ ] 2018-11-05 23:41:40: Iteration calculation: 6
[ ] 2018-11-05 23:41:40: Iteration calculation: 7
[ ] 2018-11-05 23:41:40: Iteration calculation: 8
[ ] 2018-11-05 23:41:40: Iteration calculation: 9
[ ] 2018-11-05 23:41:41: Iteration calculation: 10
[ ] 2018-11-05 23:41:41: Iteration calculation: 11
[ ] 2018-11-05 23:41:41: Iteration calculation: 12
[ ] 2018-11-05 23:41:41: Iteration calculation: 13
[ ] 2018-11-05 23:41:41: Iteration calculation: 14
[ ] 2018-11-05 23:41:41: Iteration calculation: 15
[ ] 2018-11-05 23:41:41: Iteration calculation: 16
[ ] 2018-11-05 23:41:42: Iteration calculation: 17
[ ] 2018-11-05 23:41:42: Iteration calculation: 18
[ ] 2018-11-05 23:41:42: Iteration calcul

[ ] 2018-11-05 23:42:05: Iteration calculation: 161
[ ] 2018-11-05 23:42:05: Iteration calculation: 162
[ ] 2018-11-05 23:42:05: Iteration calculation: 163
[ ] 2018-11-05 23:42:06: Iteration calculation: 164
[ ] 2018-11-05 23:42:06: Iteration calculation: 165
[ ] 2018-11-05 23:42:06: Iteration calculation: 166
[ ] 2018-11-05 23:42:06: Iteration calculation: 167
[ ] 2018-11-05 23:42:07: Iteration calculation: 168
[ ] 2018-11-05 23:42:07: Iteration calculation: 169
[ ] 2018-11-05 23:42:07: Iteration calculation: 170
[ ] 2018-11-05 23:42:07: Iteration calculation: 171
[ ] 2018-11-05 23:42:07: Iteration calculation: 172
[ ] 2018-11-05 23:42:08: Iteration calculation: 173
[ ] 2018-11-05 23:42:08: Iteration calculation: 174
[ ] 2018-11-05 23:42:08: Iteration calculation: 175
[ ] 2018-11-05 23:42:08: Iteration calculation: 176
[ ] 2018-11-05 23:42:08: Iteration calculation: 177
[ ] 2018-11-05 23:42:08: Iteration calculation: 178
[ ] 2018-11-05 23:42:09: Iteration calculation: 179
[ ] 2018-11-

# McW5 

In [11]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer

print(make_results_str(measure_all_quantities(
    p_mcw5,
    dataset,
    molecules[ind:],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-11-05 23:42:57: Iteration calculation: 0
[ ] 2018-11-05 23:42:57: Iteration calculation: 1
[ ] 2018-11-05 23:42:58: Iteration calculation: 2
[ ] 2018-11-05 23:42:58: Iteration calculation: 3
[ ] 2018-11-05 23:42:58: Iteration calculation: 4
[ ] 2018-11-05 23:42:58: Iteration calculation: 5
[ ] 2018-11-05 23:42:58: Iteration calculation: 6
[ ] 2018-11-05 23:42:59: Iteration calculation: 7
[ ] 2018-11-05 23:42:59: Iteration calculation: 8
[ ] 2018-11-05 23:42:59: Iteration calculation: 9
[ ] 2018-11-05 23:42:59: Iteration calculation: 10
[ ] 2018-11-05 23:42:59: Iteration calculation: 11
[ ] 2018-11-05 23:43:00: Iteration calculation: 12
[ ] 2018-11-05 23:43:00: Iteration calculation: 13
[ ] 2018-11-05 23:43:01: Iteration calculation: 14
[ ] 2018-11-05 23:43:01: Iteration calculation: 15
[ ] 2018-11-05 23:43:01: Iteration calculation: 16
[ ] 2018-11-05 23:43:01: Iteration calculation: 17
[ ] 2018-11-05 23:43:01: Iteration calculation: 18
[ ] 2018-11-05 23:43:02: Iteration calcul

[ ] 2018-11-05 23:43:29: Iteration calculation: 160
[ ] 2018-11-05 23:43:30: Iteration calculation: 161
[ ] 2018-11-05 23:43:30: Iteration calculation: 162
[ ] 2018-11-05 23:43:30: Iteration calculation: 163
[ ] 2018-11-05 23:43:30: Iteration calculation: 164
[ ] 2018-11-05 23:43:31: Iteration calculation: 165
[ ] 2018-11-05 23:43:31: Iteration calculation: 166
[ ] 2018-11-05 23:43:31: Iteration calculation: 167
[ ] 2018-11-05 23:43:31: Iteration calculation: 168
[ ] 2018-11-05 23:43:32: Iteration calculation: 169
[ ] 2018-11-05 23:43:32: Iteration calculation: 170
[ ] 2018-11-05 23:43:32: Iteration calculation: 171
[ ] 2018-11-05 23:43:33: Iteration calculation: 172
[ ] 2018-11-05 23:43:33: Iteration calculation: 173
[ ] 2018-11-05 23:43:33: Iteration calculation: 174
[ ] 2018-11-05 23:43:33: Iteration calculation: 175
[ ] 2018-11-05 23:43:34: Iteration calculation: 176
[ ] 2018-11-05 23:43:34: Iteration calculation: 177
[ ] 2018-11-05 23:43:34: Iteration calculation: 178
[ ] 2018-11-

In [12]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics

print("--- Iterations DAMPED ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_damping,
    p_mcw5.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-05 23:43:38: Iteration calculation: 0
[ ] 2018-11-05 23:43:38: Iteration calculation: 1
[ ] 2018-11-05 23:43:38: Iteration calculation: 2
[ ] 2018-11-05 23:43:39: Iteration calculation: 3
[ ] 2018-11-05 23:43:39: Iteration calculation: 4
[ ] 2018-11-05 23:43:39: Iteration calculation: 5
[ ] 2018-11-05 23:43:39: Iteration calculation: 6
[ ] 2018-11-05 23:43:39: Iteration calculation: 7
[ ] 2018-11-05 23:43:39: Iteration calculation: 8
[ ] 2018-11-05 23:43:39: Iteration calculation: 9
[ ] 2018-11-05 23:43:40: Iteration calculation: 10
[ ] 2018-11-05 23:43:40: Iteration calculation: 11
[ ] 2018-11-05 23:43:40: Iteration calculation: 12
[ ] 2018-11-05 23:43:40: Iteration calculation: 13
[ ] 2018-11-05 23:43:40: Iteration calculation: 14
[ ] 2018-11-05 23:43:41: Iteration calculation: 15
[ ] 2018-11-05 23:43:41: Iteration calculation: 16
[ ] 2018-11-05 23:43:41: Iteration calculation: 17
[ ] 2018-11-05 23:43:41: Iteration calculation: 18
[ ] 2018-11-05 23:43:42: Iteration calcul

[ ] 2018-11-05 23:44:07: Iteration calculation: 161
[ ] 2018-11-05 23:44:07: Iteration calculation: 162
[ ] 2018-11-05 23:44:07: Iteration calculation: 163
[ ] 2018-11-05 23:44:07: Iteration calculation: 164
[ ] 2018-11-05 23:44:07: Iteration calculation: 165
[ ] 2018-11-05 23:44:07: Iteration calculation: 166
[ ] 2018-11-05 23:44:08: Iteration calculation: 167
[ ] 2018-11-05 23:44:08: Iteration calculation: 168
[ ] 2018-11-05 23:44:08: Iteration calculation: 169
[ ] 2018-11-05 23:44:08: Iteration calculation: 170
[ ] 2018-11-05 23:44:08: Iteration calculation: 171
[ ] 2018-11-05 23:44:08: Iteration calculation: 172
[ ] 2018-11-05 23:44:08: Iteration calculation: 173
[ ] 2018-11-05 23:44:09: Iteration calculation: 174
[ ] 2018-11-05 23:44:09: Iteration calculation: 175
[ ] 2018-11-05 23:44:09: Iteration calculation: 176
[ ] 2018-11-05 23:44:09: Iteration calculation: 177
[ ] 2018-11-05 23:44:09: Iteration calculation: 178
[ ] 2018-11-05 23:44:09: Iteration calculation: 179
[ ] 2018-11-

In [13]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics

print("--- Iterations DIIS ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_diis,
    p_mcw5.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-05 23:44:13: Iteration calculation: 0
[ ] 2018-11-05 23:44:13: Iteration calculation: 1
[ ] 2018-11-05 23:44:13: Iteration calculation: 2
[ ] 2018-11-05 23:44:13: Iteration calculation: 3
[ ] 2018-11-05 23:44:13: Iteration calculation: 4
[ ] 2018-11-05 23:44:14: Iteration calculation: 5
[ ] 2018-11-05 23:44:14: Iteration calculation: 6
[ ] 2018-11-05 23:44:14: Iteration calculation: 7
[ ] 2018-11-05 23:44:14: Iteration calculation: 8
[ ] 2018-11-05 23:44:14: Iteration calculation: 9
[ ] 2018-11-05 23:44:14: Iteration calculation: 10
[ ] 2018-11-05 23:44:14: Iteration calculation: 11
[ ] 2018-11-05 23:44:15: Iteration calculation: 12
[ ] 2018-11-05 23:44:15: Iteration calculation: 13
[ ] 2018-11-05 23:44:15: Iteration calculation: 14
[ ] 2018-11-05 23:44:15: Iteration calculation: 15
[ ] 2018-11-05 23:44:15: Iteration calculation: 16
[ ] 2018-11-05 23:44:15: Iteration calculation: 17
[ ] 2018-11-05 23:44:15: Iteration calculation: 18
[ ] 2018-11-05 23:44:16: Iteration calcul

[ ] 2018-11-05 23:44:41: Iteration calculation: 160
[ ] 2018-11-05 23:44:41: Iteration calculation: 161
[ ] 2018-11-05 23:44:41: Iteration calculation: 162
[ ] 2018-11-05 23:44:42: Iteration calculation: 163
[ ] 2018-11-05 23:44:42: Iteration calculation: 164
[ ] 2018-11-05 23:44:42: Iteration calculation: 165
[ ] 2018-11-05 23:44:42: Iteration calculation: 166
[ ] 2018-11-05 23:44:42: Iteration calculation: 167
[ ] 2018-11-05 23:44:43: Iteration calculation: 168
[ ] 2018-11-05 23:44:43: Iteration calculation: 169
[ ] 2018-11-05 23:44:43: Iteration calculation: 170
[ ] 2018-11-05 23:44:43: Iteration calculation: 171
[ ] 2018-11-05 23:44:44: Iteration calculation: 172
[ ] 2018-11-05 23:44:44: Iteration calculation: 173
[ ] 2018-11-05 23:44:44: Iteration calculation: 174
[ ] 2018-11-05 23:44:44: Iteration calculation: 175
[ ] 2018-11-05 23:44:44: Iteration calculation: 176
[ ] 2018-11-05 23:44:45: Iteration calculation: 177
[ ] 2018-11-05 23:44:45: Iteration calculation: 178
[ ] 2018-11-

# Rescaled

In [14]:
from SCFInitialGuess.nn.post_processing import rescale_batch

p_rescaled = rescale_batch(p_batch, s_raw, N_ELECTRONS)

In [15]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer

print(make_results_str(measure_all_quantities(
    p_rescaled,
    dataset,
    molecules[ind:],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-11-05 23:45:37: Iteration calculation: 0
[ ] 2018-11-05 23:45:37: Iteration calculation: 1
[ ] 2018-11-05 23:45:38: Iteration calculation: 2
[ ] 2018-11-05 23:45:38: Iteration calculation: 3
[ ] 2018-11-05 23:45:38: Iteration calculation: 4
[ ] 2018-11-05 23:45:38: Iteration calculation: 5
[ ] 2018-11-05 23:45:39: Iteration calculation: 6
[ ] 2018-11-05 23:45:39: Iteration calculation: 7
[ ] 2018-11-05 23:45:39: Iteration calculation: 8
[ ] 2018-11-05 23:45:39: Iteration calculation: 9
[ ] 2018-11-05 23:45:39: Iteration calculation: 10
[ ] 2018-11-05 23:45:40: Iteration calculation: 11
[ ] 2018-11-05 23:45:40: Iteration calculation: 12
[ ] 2018-11-05 23:45:40: Iteration calculation: 13
[ ] 2018-11-05 23:45:42: Iteration calculation: 14
[ ] 2018-11-05 23:45:42: Iteration calculation: 15
[ ] 2018-11-05 23:45:42: Iteration calculation: 16
[ ] 2018-11-05 23:45:42: Iteration calculation: 17
[ ] 2018-11-05 23:45:43: Iteration calculation: 18
[ ] 2018-11-05 23:45:43: Iteration calcul

[ ] 2018-11-05 23:46:19: Iteration calculation: 160
[ ] 2018-11-05 23:46:20: Iteration calculation: 161
[ ] 2018-11-05 23:46:20: Iteration calculation: 162
[ ] 2018-11-05 23:46:20: Iteration calculation: 163
[ ] 2018-11-05 23:46:20: Iteration calculation: 164
[ ] 2018-11-05 23:46:21: Iteration calculation: 165
[ ] 2018-11-05 23:46:21: Iteration calculation: 166
[ ] 2018-11-05 23:46:21: Iteration calculation: 167
[ ] 2018-11-05 23:46:21: Iteration calculation: 168
[ ] 2018-11-05 23:46:21: Iteration calculation: 169
[ ] 2018-11-05 23:46:22: Iteration calculation: 170
[ ] 2018-11-05 23:46:22: Iteration calculation: 171
[ ] 2018-11-05 23:46:22: Iteration calculation: 172
[ ] 2018-11-05 23:46:22: Iteration calculation: 173
[ ] 2018-11-05 23:46:22: Iteration calculation: 174
[ ] 2018-11-05 23:46:23: Iteration calculation: 175
[ ] 2018-11-05 23:46:23: Iteration calculation: 176
[ ] 2018-11-05 23:46:23: Iteration calculation: 177
[ ] 2018-11-05 23:46:23: Iteration calculation: 178
[ ] 2018-11-

In [16]:
9 / 101

0.0891089108910891

In [17]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics

print("--- Iterations DAMPED ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_damping,
    p_rescaled.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-05 23:46:29: Iteration calculation: 0
[ ] 2018-11-05 23:46:29: Iteration calculation: 1
[ ] 2018-11-05 23:46:29: Iteration calculation: 2
[ ] 2018-11-05 23:46:29: Iteration calculation: 3
[ ] 2018-11-05 23:46:29: Iteration calculation: 4
[ ] 2018-11-05 23:46:29: Iteration calculation: 5
[ ] 2018-11-05 23:46:30: Iteration calculation: 6
[ ] 2018-11-05 23:46:30: Iteration calculation: 7
[ ] 2018-11-05 23:46:30: Iteration calculation: 8
[ ] 2018-11-05 23:46:30: Iteration calculation: 9
[ ] 2018-11-05 23:46:30: Iteration calculation: 10
[ ] 2018-11-05 23:46:30: Iteration calculation: 11
[ ] 2018-11-05 23:46:30: Iteration calculation: 12
[ ] 2018-11-05 23:46:31: Iteration calculation: 13
[ ] 2018-11-05 23:46:31: Iteration calculation: 14
[ ] 2018-11-05 23:46:31: Iteration calculation: 15
[ ] 2018-11-05 23:46:31: Iteration calculation: 16
[ ] 2018-11-05 23:46:31: Iteration calculation: 17
[ ] 2018-11-05 23:46:31: Iteration calculation: 18
[ ] 2018-11-05 23:46:31: Iteration calcul

[ ] 2018-11-05 23:46:55: Iteration calculation: 161
[ ] 2018-11-05 23:46:55: Iteration calculation: 162
[ ] 2018-11-05 23:46:55: Iteration calculation: 163
[ ] 2018-11-05 23:46:56: Iteration calculation: 164
[ ] 2018-11-05 23:46:56: Iteration calculation: 165
[ ] 2018-11-05 23:46:56: Iteration calculation: 166
[ ] 2018-11-05 23:46:56: Iteration calculation: 167
[ ] 2018-11-05 23:46:56: Iteration calculation: 168
[ ] 2018-11-05 23:46:57: Iteration calculation: 169
[ ] 2018-11-05 23:46:57: Iteration calculation: 170
[ ] 2018-11-05 23:46:57: Iteration calculation: 171
[ ] 2018-11-05 23:46:57: Iteration calculation: 172
[ ] 2018-11-05 23:46:57: Iteration calculation: 173
[ ] 2018-11-05 23:46:57: Iteration calculation: 174
[ ] 2018-11-05 23:46:58: Iteration calculation: 175
[ ] 2018-11-05 23:46:58: Iteration calculation: 176
[ ] 2018-11-05 23:46:58: Iteration calculation: 177
[ ] 2018-11-05 23:46:58: Iteration calculation: 178
[ ] 2018-11-05 23:46:58: Iteration calculation: 179
[ ] 2018-11-

In [18]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics

print("--- Iterations DIIS ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_diis,
    p_rescaled.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-11-05 23:47:02: Iteration calculation: 0
[ ] 2018-11-05 23:47:02: Iteration calculation: 1
[ ] 2018-11-05 23:47:02: Iteration calculation: 2
[ ] 2018-11-05 23:47:02: Iteration calculation: 3
[ ] 2018-11-05 23:47:03: Iteration calculation: 4
[ ] 2018-11-05 23:47:03: Iteration calculation: 5
[ ] 2018-11-05 23:47:03: Iteration calculation: 6
[ ] 2018-11-05 23:47:03: Iteration calculation: 7
[ ] 2018-11-05 23:47:03: Iteration calculation: 8
[ ] 2018-11-05 23:47:03: Iteration calculation: 9
[ ] 2018-11-05 23:47:04: Iteration calculation: 10
[ ] 2018-11-05 23:47:04: Iteration calculation: 11
[ ] 2018-11-05 23:47:04: Iteration calculation: 12
[ ] 2018-11-05 23:47:04: Iteration calculation: 13
[ ] 2018-11-05 23:47:04: Iteration calculation: 14
[ ] 2018-11-05 23:47:04: Iteration calculation: 15
[ ] 2018-11-05 23:47:04: Iteration calculation: 16
[ ] 2018-11-05 23:47:05: Iteration calculation: 17
[ ] 2018-11-05 23:47:05: Iteration calculation: 18
[ ] 2018-11-05 23:47:05: Iteration calcul

[ ] 2018-11-05 23:47:26: Iteration calculation: 160
[ ] 2018-11-05 23:47:26: Iteration calculation: 161
[ ] 2018-11-05 23:47:27: Iteration calculation: 162
[ ] 2018-11-05 23:47:27: Iteration calculation: 163
[ ] 2018-11-05 23:47:27: Iteration calculation: 164
[ ] 2018-11-05 23:47:27: Iteration calculation: 165
[ ] 2018-11-05 23:47:27: Iteration calculation: 166
[ ] 2018-11-05 23:47:27: Iteration calculation: 167
[ ] 2018-11-05 23:47:27: Iteration calculation: 168
[ ] 2018-11-05 23:47:27: Iteration calculation: 169
[ ] 2018-11-05 23:47:28: Iteration calculation: 170
[ ] 2018-11-05 23:47:28: Iteration calculation: 171
[ ] 2018-11-05 23:47:28: Iteration calculation: 172
[ ] 2018-11-05 23:47:28: Iteration calculation: 173
[ ] 2018-11-05 23:47:28: Iteration calculation: 174
[ ] 2018-11-05 23:47:28: Iteration calculation: 175
[ ] 2018-11-05 23:47:28: Iteration calculation: 176
[ ] 2018-11-05 23:47:28: Iteration calculation: 177
[ ] 2018-11-05 23:47:29: Iteration calculation: 178
[ ] 2018-11-

# Rescaled diag 

In [19]:
from SCFInitialGuess.nn.post_processing import rescale_diag_batch

p_rescaled_diag = rescale_diag_batch(p_batch, s_raw, N_ELECTRONS)

In [20]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer

print(make_results_str(measure_all_quantities(
    p_rescaled_diag,
    dataset,
    molecules[ind:],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-11-05 23:48:08: Iteration calculation: 0
[ ] 2018-11-05 23:48:09: Iteration calculation: 1
[ ] 2018-11-05 23:48:09: Iteration calculation: 2
[ ] 2018-11-05 23:48:10: Iteration calculation: 3
[ ] 2018-11-05 23:48:10: Iteration calculation: 4
[ ] 2018-11-05 23:48:11: Iteration calculation: 5
[ ] 2018-11-05 23:48:11: Iteration calculation: 6
[ ] 2018-11-05 23:48:11: Iteration calculation: 7
[ ] 2018-11-05 23:48:12: Iteration calculation: 8
[ ] 2018-11-05 23:48:12: Iteration calculation: 9
[ ] 2018-11-05 23:48:12: Iteration calculation: 10
[ ] 2018-11-05 23:48:12: Iteration calculation: 11
[ ] 2018-11-05 23:48:13: Iteration calculation: 12
[ ] 2018-11-05 23:48:13: Iteration calculation: 13
[ ] 2018-11-05 23:48:14: Iteration calculation: 14
[ ] 2018-11-05 23:48:14: Iteration calculation: 15
[ ] 2018-11-05 23:48:15: Iteration calculation: 16
[ ] 2018-11-05 23:48:15: Iteration calculation: 17
[ ] 2018-11-05 23:48:15: Iteration calculation: 18
[ ] 2018-11-05 23:48:16: Iteration calcul

[ ] 2018-11-05 23:49:03: Iteration calculation: 160
[ ] 2018-11-05 23:49:03: Iteration calculation: 161
[ ] 2018-11-05 23:49:03: Iteration calculation: 162
[ ] 2018-11-05 23:49:03: Iteration calculation: 163
[ ] 2018-11-05 23:49:04: Iteration calculation: 164
[ ] 2018-11-05 23:49:04: Iteration calculation: 165
[ ] 2018-11-05 23:49:04: Iteration calculation: 166
[ ] 2018-11-05 23:49:04: Iteration calculation: 167
[ ] 2018-11-05 23:49:05: Iteration calculation: 168
[ ] 2018-11-05 23:49:05: Iteration calculation: 169
[ ] 2018-11-05 23:49:05: Iteration calculation: 170
[ ] 2018-11-05 23:49:06: Iteration calculation: 171
[ ] 2018-11-05 23:49:06: Iteration calculation: 172
[ ] 2018-11-05 23:49:06: Iteration calculation: 173
[ ] 2018-11-05 23:49:07: Iteration calculation: 174
[ ] 2018-11-05 23:49:07: Iteration calculation: 175
[ ] 2018-11-05 23:49:07: Iteration calculation: 176
[ ] 2018-11-05 23:49:08: Iteration calculation: 177
[ ] 2018-11-05 23:49:08: Iteration calculation: 178
[ ] 2018-11-